In [1]:
import numpy as np
import pandas as pd

In [3]:
class chain_ladder():
    def __init__(self,df):
        self.column_indices = {c: i for i, c in enumerate(df.columns)}
        self.pay00i = self.column_indices['Pay00']
        self.df = df
    
    def make_triangle_data(self):
        triangle = pd.DataFrame(columns = ['RepDel','AY','RelDevYear','claim_incurred','link_ratio'])
        for rep_del in range(12):
            years = 12-rep_del
            start_i = self.pay00i + rep_del
            for ay in range(1,years+1):
                tmp = self.df.loc[(self.df['AY']==ay)&(self.df['RepDel']==rep_del)]
                claim_incurred = 0
                for rel_dev in range(years-ay+1):
                    index = start_i + rel_dev
                    claim_incurred_new = claim_incurred + tmp.iloc[:,index].mean()
                    if claim_incurred != 0:
                        link_ratio = claim_incurred_new/claim_incurred
                    else:
                        link_ratio = np.NaN
                    claim_incurred = claim_incurred_new
                    new_row = {'RepDel':rep_del, 'AY':ay, 'RelDevYear':rel_dev, 'claim_incurred':claim_incurred, 
                          'link_ratio': link_ratio}
                    triangle = triangle.append(new_row, ignore_index = True)
        return triangle
    
    def get_average_link_ratio(self, triangle):
        average = pd.DataFrame(columns = ['RepDel','RelDevYear','link'])
        for rep_del in range(11):
            for rel_dev in range(1,12-rep_del):
                avg_link = triangle.loc[(triangle['RepDel']==rep_del)&(triangle['RelDevYear']==rel_dev)].link_ratio.mean()
                new_row = {'RepDel':rep_del,'RelDevYear':rel_dev,'link': avg_link}
                average = average.append(new_row, ignore_index = True)
        return average
    
    def chain_ladder_average_UC(self):
        triangle = self.make_triangle_data()
        average_links = self.get_average_link_ratio(triangle)
    
        average_UC = pd.DataFrame(columns = ['RepDel','AY','avg_UC'])
    
        for rep_del in range(12):
            years = 12-rep_del
            #start_i = self.pay00i + rep_del
            for ay in range(1,years+1):
                max_dev = years - ay
                UC = float(triangle.loc[(triangle['RepDel']==rep_del)&
                              (triangle['AY']==ay)&
                             (triangle['RelDevYear']==max_dev)].claim_incurred)
                while max_dev < (years-1):
                    factor = float(average_links.loc[(average_links['RepDel']==rep_del)&
                                                (average_links['RelDevYear']==(max_dev+1))].link)
                    UC = UC*factor
                    max_dev += 1
            
                new_row = {'RepDel':rep_del,'AY':ay,'avg_UC': UC}
                average_UC = average_UC.append(new_row, ignore_index = True)
        
        exposures = []
        for rep_del in range(12):
            years = 12-rep_del
            for ay in range(1,years+1):
                exposures.append(len(self.df.loc[(self.df['RepDel']==rep_del)&(self.df['AY']==ay)]))
        
        average_UC['Exposure'] = exposures
    
        return average_UC